In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets , transforms
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from keras.layers import Input, Convolution3D, MaxPooling3D, UpSampling3D
from pyexpat import model
from torch.utils.data import Dataset, DataLoader, random_split
import nibabel as nib
import numpy as np
import os
import cv2
import torch.optim
import torch.optim.lr_scheduler as ll


c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
import shutil
# DONT RUN THIS CELL
path = r"D:\DELL\Documents\final_output"
output_dir = r'D:\DELL\Documents\filtered_data'
items = os.listdir(path)
clean_list =[]
for i,item in enumerate(items):
    item_name = os.path.join(path, items[i])
    item_data = nib.load(item_name)
    item_array = item_data.get_fdata()
    item_array = item_array[0:128]
    item_array = torch.tensor(item_array)
    if(item_array.shape == torch.Size([128, 240, 256])):
        #clean_list.append(item_array)
        shutil.move(item_name, os.path.join(output_dir, item))



KeyboardInterrupt: 

In [25]:
for item in clean_list:
    print(item.shape)

torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240, 256])
torch.Size([128, 240

In [3]:
output_dir = r'D:\DELL\Documents\filtered_data'

class Patient_Dataset(Dataset):
  def __init__(self, transform = None):
      self.root_dir = output_dir
      self.transform = transform
      self.nifti_files = os.listdir(output_dir)
      self.transforms = transforms

  def __len__(self):
    return len(self.nifti_files)

  def __getitem__(self, idx):
     nifti_name = os.path.join(self.root_dir,self.nifti_files[idx])
     nifti_data = nib.load(nifti_name)
     nifti_array = nifti_data.get_fdata()
     nifti_array = nifti_array[0:128]
     
    # nifti_array = clean_list[idx]

        # Resize the NIfTI data


        # Stack the resized slices
      #resized_data = np.stack(resized_data)

     if self.transform:
          nifti_array = self.transform(nifti_array)

     else:
          nifti_array = torch.tensor(nifti_array)
          nifti_array = nifti_array.unsqueeze(0)
     return nifti_array



In [4]:
data_transform = transforms.Compose([
    transforms.ToTensor()   # there might be something with this function that switchs the slice index with the hieght index
])
dataset = Patient_Dataset( )#,transform=data_transform)
data_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)
print(len(dataset))

'''for images in data_loader:

    # Print the minimum and maximum values of the images in the batch
    print(torch.min(images), torch.max(images))'''

106


'for images in data_loader:\n\n    # Print the minimum and maximum values of the images in the batch\n    print(torch.min(images), torch.max(images))'

In [9]:
i =0
j=0
t =0
for data in dataset:
    if data.shape == torch.Size([128, 256, 256]):
        i+=1
    elif data.shape == torch.Size([128, 240, 256]):
        j+=1
    elif data.shape == torch.Size([128, 192, 192]):
        dataset.__delattr__
        t+=1
    else:
        print("a weird data shape")  
        print(data.shape)

print(i)
print(j)
print(t)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1412\3658009883.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  nifti_array = torch.tensor(nifti_array)


0
106
0


In [5]:
sample = dataset[3]  # Get the first sample from the dataset
print("Resized data shape:", sample.shape)

Resized data shape: torch.Size([1, 128, 240, 256])


In [10]:
class Encoder3D(nn.Module):
    def __init__(self):
        super(Encoder3D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1),  # Output: 8x28x28x28
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),  # Output: 8x14x14x14
            nn.Conv3d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),  # Output: 16x14x14x14  avoid maxpooling
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),  # Output: 16x7x7x7
            nn.Conv3d(in_channels=16, out_channels=16, kernel_size=3, stride=2, padding=2),  # Output: 16x7x7x7    research leaky relu gelu
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=1),  # Output: 16x6x6x6     need normalization maybe random dropout or batch normalization and
            # note : we also changed the stride to so the output is an integer
            #nn.Conv3d(in_channels=16, out_channels=3, kernel_size=3, stride=1, padding=1)  # Output: 3x4x4x4

            # for calculating the output of a MaxPool layer we use (n - w)/s +1
            # n = input size
            # w = kernel size
            # s = stride size

            # for calculating the output size of a convLayer we use (((W - K + 2P) / S) + 1)
            # n = input size
            # w = kernel size
            # s = stride size
            # p = padding
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

# Create an instance
encoder_3d = Encoder3D()

x_3d = torch.randn(1,1,240, 256, 128)
#x_3d = torch.randn(20, 1, 28, 28, 28)
y_3d = encoder_3d(x_3d)
print(x_3d.shape)
print(y_3d.shape)  # torch.Size([20, 3, 4, 4, 4]) the same as the last conv layer

class Decoder3D(nn.Module):
    def __init__(self):
        super(Decoder3D, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(in_channels=16, out_channels=16, kernel_size=3, stride=2, padding=1,output_padding=1,dilation=1),
            #nn.ConvTranspose3d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=0),  # Output: 16x9x9x9
            nn.ReLU(),

            nn.ConvTranspose3d(in_channels=16, out_channels=8, kernel_size=3, stride=2, padding=1, output_padding=1,dilation=1),  # Output: 8x18x18x18
            nn.ReLU(),

            nn.ConvTranspose3d(in_channels=8, out_channels=1, kernel_size=3, stride=2, padding=2, output_padding= 1, dilation=2),  # Output: 4x36x36x36
            nn.ReLU()
            #nn.ConvTranspose3d(in_channels=4, out_channels=1, kernel_size=3, stride=1, padding=5)  # Output: 1x28x28x28

            # the equation for the output of transpose convLayers is (W−1)*S − 2×P + dilation ×(K−1) + output_padding +1
            # W = Input size
            # K = Size of kernels (Filter size)
            # S = Stride
            # P = Padding
            # By
            # default, stride = 1, padding = 0, dilation = 1, and output_padding = 0

        )

    def forward(self, x):
        x = self.decoder(x)
        return x

# Create an instance
decoder_3d = Decoder3D()

x_decoded_3d = decoder_3d(y_3d) # y_3d is the encoded output from the encoder
print(x_decoded_3d.shape)  # torch.Size([20, 1, 28, 28, 28])

torch.Size([1, 1, 240, 256, 128])
torch.Size([1, 16, 30, 32, 16])
torch.Size([1, 1, 240, 256, 128])


In [7]:
pip install torchsummary

In [7]:
#from torchsummary import summary

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 30, kernel_size=5, padding=2),
            nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, padding=1),
            nn.Conv3d(30, 60, kernel_size=5, padding=2),
            nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, padding=1)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.ConvTranspose3d(60, 30, kernel_size=5, padding=3),
            nn.ReLU(True),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.ConvTranspose3d(30,  1, kernel_size=5, padding=2),

        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

sample_tensor = torch.randn( 1,128, 240, 256)

autoencoder = Autoencoder()

#reconstructed_tensor = autoencoder(sample_tensor)

#print("Shape of reconstructed tensor:", reconstructed_tensor.shape)
#print(autoencoder)
#summary(autoencoder, ( 1,128, 240, 256))

In [8]:
class Autoencoder3D(nn.Module):
    def __init__(self):
        super(Autoencoder3D, self).__init__()
        self.encoder = Encoder3D()
        self.decoder = Decoder3D()

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [9]:
import torch.optim as optim


autoencoder = Autoencoder()
criterion = nn.MSELoss()  # Mean Squared Error loss maybe mae will work better
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [10]:
import torch.optim
import torch.optim.lr_scheduler as ll

scheduler = ll.ReduceLROnPlateau(optimizer, 'min',factor=0.1,patience=3 )

In [11]:
num_epochs = 10
outputs =[]
for epoch in range(num_epochs):
    for idx, img in enumerate(data_loader):
        img = img.float()  # Convert input tensor to float

        recon = autoencoder(img)
        loss = criterion(recon, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step(loss)
    print(f'Epoch: {epoch+1}, Iteration: {idx+1}, Loss: {loss.item():.4f}')

    outputs.append((epoch, img, recon))


Epoch: 1, Iteration: 106, Loss: 167.0439
Epoch: 2, Iteration: 106, Loss: 422.9420
Epoch: 3, Iteration: 106, Loss: 250.7212
Epoch: 4, Iteration: 106, Loss: 378.6319
Epoch: 5, Iteration: 106, Loss: 213.4715
Epoch: 6, Iteration: 106, Loss: 72.0956
Epoch: 7, Iteration: 106, Loss: 89.6297
Epoch: 8, Iteration: 106, Loss: 502.9212
Epoch: 9, Iteration: 106, Loss: 188.3651
Epoch: 10, Iteration: 106, Loss: 238.4381


In [20]:
print("Length of dataset:", len(dataset))


Length of dataset: 172


In [1]:
num_epochs = 10
outputs =[]
for epoch in range(num_epochs):
    for img in data_loader:
        img = img.float()  # Convert input tensor to float

        recon = autoencoder(img)
        loss = criterion(recon,img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch+1} , loss {loss.item():.4f}')
    outputs.append((epoch,img,recon))

NameError: name 'data_loader' is not defined